In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

aspects = ['bit', 'domain']

from colors import color

Using TensorFlow backend.


[]


In [ ]:
vec = cPickle.load(open('../../yelpdata/total_vec_120K_embed.p', 'rb'))

In [ ]:
len(vec.word2idx)

In [3]:
import pandas as pd
df = pd.read_csv('../../yelpdata/total_data_120K.csv')

In [4]:
ds = df
from sklearn.model_selection import train_test_split
ds['bit'] = ds['stars'].apply(lambda x : x > 3.0)
train_idxs, val_idxs = train_test_split(ds.index, stratify=ds[['bit', 'domain']], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
model = load_model('../store/weights/food/WedDec614:45:092017final1/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})

In [6]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs, 
                          [model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).layers[17].output])

In [7]:
pool_cdsr = {}
X_abs = vec.X[val_idxs]
from sklearn.preprocessing import normalize


for aspect in aspects :
    pool_cdsr[aspect] = []
    bs = 1000
    for i in range(int(np.floor(len(X_abs)/bs)) + 1) :
        val = model_c1[aspect]([X_abs[i*bs:(i+1)*bs]])[0]
        val = val.mean(axis=-1)
        val = np.dstack((X_abs[i*bs:(i+1)*bs], val))
        pool_cdsr[aspect].append(val)
    pool_cdsr[aspect] = np.concatenate(pool_cdsr[aspect])
    print pool_cdsr[aspect].shape

filter_aspect = {}
for aspect in aspects :
    filter_aspect[aspect] = pool_cdsr[aspect]

(12709, 462, 2)
(12709, 462, 2)


In [8]:
filter_aspect['bit'].shape

(12709, 462, 2)

In [9]:
activated_words = {}
for aspect in aspects :
    filters = filter_aspect[aspect]
    word_activations = np.zeros((vec.vocab_size, 1))
    word2filter = filters.reshape(-1, filters.shape[-1])
    print word2filter.shape
    for word in word2filter :
        word_activations[int(word[0]), 0] = word[1]
    activated_words[aspect] = word_activations
    
joint = np.concatenate([activated_words[aspect] for aspect in aspects], axis=1)

(5871558, 2)
(5871558, 2)


In [10]:
activated_words['bit'].shape

(10002, 1)

In [ ]:
top_words_df = pd.DataFrame()
top_words_size = 20
for aspect in aspects :
    popwords = activated_words[aspect].flatten()
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    print [str(v) for k, v in vec.idx2word.items() if k in top_words]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    
    top_words_df[aspect] = top_words
top_words_df

In [24]:
from random import randint
for m in range(50):
    s = randint(0, len(filter_aspect[aspects[0]])-1)
    gates = {}
    for aspect in aspects :
        gate0 = filter_aspect[aspect][s, :, 1]
        nz = sum([1 for i in filter_aspect[aspect][s, :, 0] if i == 0])
        
        gate0 = np.convolve(gate0, [1./3 for _ in range(3)], mode='same')
        gate0 = (gate0 - gate0[nz:].mean(axis=0))#/ gate0[nz:].std(axis=0)
        gate0 = (gate0 - gate0[nz:].min(axis=0)) / (gate0[nz:].max(axis=0) - gate0[nz:].min(axis=0))
        gate0 = gate0.clip(0, 1)


        gates[aspect] = gate0


    X_words = [vec.idx2word[int(i)] for i in filter_aspect[aspect][s, :, 0]]
    for v, aspect in enumerate(aspects) :
        for i, j in zip(X_words, gates[aspect]) :
            c = [255, 255, 255]
            for k in range(3) :
                if k != v :
                    c[k] = int((1 - j)*255)
            c[1], c[2] = c[2], c[1]
            if i != '[0]' :
                print color(i, bg=c),
        print ""
    print "--------------------------------------------------------------------------------------"+str(m)

we chose the unk hotel after reading trip advisor reviews . we wanted somewhere that was within walking distance of the major sights . previous reviews suggested that the beds were extremely comfortable , and that unk it for us ! ! the hotel decor is unk black and white - walls , sheets , wallpaper , tiles . the impression is of a hotel that has recently been updated . we did n't use the swimming pool on the unk floor . we did n't eat in the hotel , except the free crisps in the mini - bar . we ate in different restaurants / cafes every day and were never dissapointed - all within walking distances . because of the great location we were able to walk everywhere we wanted , and did n't use public transport once . all in all we were extremely happy with our stay here , but since we left for barcelona some poor reviews have appeared on trip advisor for unk hotel . i 'm afraid that we have to agree with the surly , unfriendly complaints with regard to the person that checked us in . there 

 the food is pretty average , kind of exactly what you would expect from this type of place .   we shared bruschetta which was massive and very good although the bread could have slightly more toasted .   i particularly enjoyed the addition of unk olives .   we also shared the ceasar salad which could have used more garlic but was fresh and not over dressed . 

 the mains were the worst of the meal .   my friend has the seafood unk . the pasta was well cooked but the seafood was over cooked .    i had the unk , which was spicy as requested but not good .   all the proteins were way over cooked as was the rice and veg .  

 i find their menu pretty limited and would n't go back for mains but would go back for apps .   

 they had a decent wine by the glass selection . 
i felt compelled to write this review because some of the other reviews are so negative , specifically about their service .  

 i met an old friend for dinner here last night at her suggestion and to be honest , i was un

bourbon or canal so at least our room was very quiet . however , with it being an interior room ( there was a small window ) we think that it contributed to the room being humid and warm . three of us stayed in the room and we all left with a sore throat because of the room temp / air circulation . next thing we wanted to do when we got there was take showers , and there were no wash cloths ! the entire time we had to ask the housekeeping staff to bring us washcloths . i called the front desk to complain and they said we should have them , but every time they cleaned out room they only left us hand towels and regular towels . oh and the bathroom is tiny , so is the shower -- barely enough room to move around . the restaurant in the lobby was so - so and way overpriced . you can head down one of the side streets off bourbon and get the same food for much less , guess they can charge that because of the location . also , do nt even think about ordering room service . we tried on qqq diff

we had no problems at all . complaints : all the destination weddings and wedding guests at the resort during our second weekend was slightly obnoxious . it would have been nicer and quieter without them running around and partying . overall , the resort did a good job at keeping the partying and unk of these guests minimal . the resort lobby and beach front restaurants lack of air conditioning . i was miserable each evening from the heat and humidity at dinner or having drinks in the lobby after spending the day in the sun . no cash atm at the resort . you can get cash from the front desk though via cash advance on your credit card . random tips : take plenty of cash . us dollars are unk unk everywhere at a exchange rate of $ qqq pesos to $ qqq us dollar . credit cards are accepted most , but not all places ( unlike in the us ) so to take advantage of some things , you need cash . i used credit , debit and cash and had one of my credit card # s stolen and used for unk purchases until 

well first of all , when we got there for a unk conference , i was unsure about the hotel because of some of the reviews on trip advisor . i was in for a nice surprise ! the check - in lady was really nice and pleasant , and the parking ticket was n't as much as some of the reviews said it would be ! i asked the lady if we could have a bay - view room , and she gave us one without even charging extra , and we had a perfect view of seaport village and the unk ! besides that , the grand mall was only a qqq minute walk away ! the pool was clean , and the unk were outstanding ! also , the poolside snack bar had awesome bartenders , and the waitresses were very nice ! the showers were a little dirty , but hey , what can you expect from a shower that has been used before ! the beds , were a different story , as for falling asleep on them was like sleeping on a rock ! the unk in seaport village were very pricy , but for good service and food ! back to the rooms , they were very nice , and the

i stayed at qqq paddington for qqq nights last july during a trip with friends . even though we booked two twin rooms we arrived there to find out that in this hotel only double rooms are available ( not a good start i thought ) . before checking in we were asked to pay the whole amount of money for our stay upfront which was ok but kind of strange . the hotel is actually two separate buildings with different entrance . our rooms were not in the main building so we had to go out to unk road ( a quite busy avenue ) and then enter the main unk everytime we wanted to visit the reception table or have breakfast . breakfast itself was disappointing but after what we had already seen we were n't surprised . rather poor , limited choices , we eventually gave it a miss and went out for breakfast . the worst aspect of this hotel though is the rooms . it was the smallest hotel room i 've ever stayed in . we could barely open our baggage . bathroom was equally small and uncomfortable . there was 

i was in jam and needed a place for qqq nights in hong kong , i was already there and booked via expedia from an internet cafe on hong kong island . the location is not great , the hotel looks nothing like the pictures . front desk area very small , and very unorganized . was told i had a great view upon check - in on the the qqq floor . i overlooked a freeway on - ramp and the windows in the room are not unk at all . the room itself was about as small and dingy as a unk from let 's say around qqq and like so many others have said , very dirty and walls paper thin . it was so small that my boyfriend and i actually bumped into each other quite a lot . i asked to see another room immediately but was told it would cost $ qqq us more a night to upgrade , and that room was much nicer , but not worth the extra money . if you want to be close to the " unk side of unk . , this is the hotel for you . lots of shops that sell a lot of nothing , not near anything except the temple street night mar